# Regression Week 5: LASSO Assignment 2

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:

- Write a function to normalize features
- Implement coordinate descent for LASSO
- Explore effects of L1 penalty

In [1]:
import pandas as pd

dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

In [2]:
import numpy as np

In [3]:
def get_numpy_data(data_frame, features, output):

    features_matrix = np.array([np.ones(len(data_frame))] + [data_frame[feature] for feature in features])

    output_array = np.array(data_frame[output])
    
    return features_matrix.T, output_array

In [4]:
def predict_outcome(feature_matrix, weights):
    return feature_matrix.dot(weights)

In [5]:
def normalize_features(features):
    
    norms = np.linalg.norm(features, axis=0)
    normalized_features = features / norms
    
    return normalized_features, norms

## Effect of L1 penalty

9. Consider a simple model with 2 features: ‘sqft_living’ and ‘bedrooms’. The output is ‘price’.

- First, run get_numpy_data() (or equivalent) to obtain a feature matrix with 3 columns (constant column added). Use the entire ‘sales’ dataset for now.
- Normalize columns of the feature matrix. Save the norms of original features as ‘norms’.
- Set initial weights to [1,4,1].
- Make predictions with feature matrix and initial weights.
- Compute values of ro[i], where

In [6]:
features, output = get_numpy_data(sales, ['sqft_living', 'bedrooms'], 'price')

normalized_features, norms = normalize_features(features)

initial_weights = np.array([1,4,1])

prediction = np.dot(normalized_features, initial_weights)

ro = []

for i in range(len(initial_weights)):
    ro.append(np.sum(normalized_features[:,i] * (output - prediction + initial_weights[i] * normalized_features[:,i])))

In [7]:
ro

[79400300.014522895, 87939470.823251754, 80966698.66623947]

       ┌ (ro[i] + lambda/2)    if ro[i] < -lambda/2
w[i] = ├ 0                     if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)    if ro[i] > lambda/2

In [8]:
87939470 * 2, 80966698 * 2

(175878940, 161933396)

# Single Coordinate Descent Step

12. Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [9]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = np.dot(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.sum(feature_matrix[:,i] * (output - prediction + weights[i] * feature_matrix[:,i]))
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [10]:
# should print 0.425558846691
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.425558846691


# Cyclical coordinate descent

## 13. Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each i§mteration:

- As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
- After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to the previous step.
- Return weights

The function should accept the following parameters:

- Feature matrix
- Output array
- Initial weights
- L1 penalty
- Tolerance

In [20]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    
    weights = [w for w in initial_weights]

    maximum_change = None
    
    while(maximum_change == None or maximum_change >= tolerance):
        maximum_change = None

        for i in range(len(initial_weights)):
            new_weight = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            delta = abs(new_weight - weights[i])
            if (maximum_change == None or delta > maximum_change): maximum_change = delta
            weights[i] = new_weight
    
    return weights    
    

In [21]:
feature_matrix = normalized_features
l1_penalty = 1e7
tolerance = 1.0
initial_weights = np.array([0,0,0])
weights = lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance)
weights

[21624997.959519088, 63157247.207889557, 0.0]

In [22]:
rss = lambda y, y_hat: np.dot((y - y_hat).T, (y - y_hat))

In [23]:
rss(output, np.dot(feature_matrix, weights))

1630492476715387.0

In [24]:
testing = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)


In [25]:
feature_list = [
    'bedrooms', 
    'bathrooms', 
    'sqft_living', 
    'sqft_lot', 
    'floors', 
    'waterfront', 
    'view', 
    'condition', 
    'grade',
    'sqft_above', 
    'sqft_basement', 
    'yr_built', 
    'yr_renovated'
]
training_features, training_output = get_numpy_data(training, feature_list, 'price')
training_features, norms = normalize_features(training_features)


First, learn the weights with l1_penalty=1e7, on the training data. Initialize weights to all zeros, and set the tolerance=1. Call resulting weights’ weights1e7’, you will need them later.

In [26]:
weights1e7 = lasso_cyclical_coordinate_descent(
    feature_matrix=training_features,
    output=training_output,
    initial_weights=np.zeros(len(feature_list) + 1),
    l1_penalty=1e7, 
    tolerance=1.0
)
weights1e7

[24429600.234403118,
 0.0,
 0.0,
 48389174.771548964,
 0.0,
 0.0,
 3317511.2149216533,
 7329961.8117142543,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [38]:
print(np.array(['intercept'] + feature_list)[np.array(weights1e7) != 0.])

['intercept' 'sqft_living' 'waterfront' 'view']


In [39]:
weights1e8 = lasso_cyclical_coordinate_descent(
    feature_matrix=training_features,
    output=training_output,
    initial_weights=np.zeros(len(feature_list) + 1),
    l1_penalty=1e8, 
    tolerance=1.0
)
weights1e8, print(np.array(['intercept'] + feature_list)[np.array(weights1e8) != 0.])

['intercept']


([71114625.714887023,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 None)

In [40]:
weights1e4 = lasso_cyclical_coordinate_descent(
    feature_matrix=training_features,
    output=training_output,
    initial_weights=np.zeros(len(feature_list) + 1),
    l1_penalty=1e4, 
    tolerance=5e5
)
weights1e4, print(np.array(['intercept'] + feature_list)[np.array(weights1e4) != 0.])

['intercept' 'bedrooms' 'bathrooms' 'sqft_living' 'sqft_lot' 'floors'
 'waterfront' 'view' 'condition' 'grade' 'sqft_above' 'sqft_basement'
 'yr_built' 'yr_renovated']


([78564738.341567621,
  -22097398.92430532,
  12791071.872785171,
  93808088.092811927,
  -2013172.7570495396,
  -4219184.9326501433,
  6482842.8175350642,
  7127408.5348068858,
  5001664.8546964023,
  14327518.437140508,
  -15770959.152373968,
  -5159591.2221314702,
  -84495341.768436402,
  2824439.4970368347],
 None)

In [42]:
testing = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)

feature_list = [
    'bedrooms', 
    'bathrooms', 
    'sqft_living', 
    'sqft_lot', 
    'floors', 
    'waterfront', 
    'view', 
    'condition', 
    'grade',
    'sqft_above', 
    'sqft_basement', 
    'yr_built', 
    'yr_renovated'
]
test_features, test_output = get_numpy_data(testing, feature_list, 'price')

normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms
normalized_weights1e4 = weights1e4 / norms

print(normalized_weights1e7[3])

161.317457646


In [45]:
print(rss(test_output, np.dot(test_features, normalized_weights1e7)))
print(rss(test_output, np.dot(test_features, normalized_weights1e8)))
print(rss(test_output, np.dot(test_features, normalized_weights1e4)))

2.7596207592e+14
5.37166151497e+14
2.28459958971e+14
